### **Packages**

In [ ]:
library(survey)
library(dplyr)
library(haven)

### **Data**

In [31]:
raw.data <- read_sav("../Vaccine_dropout_mz/Data/IDS_2022_MZKR81FL.SAV")

### **Measles I Recode**

In [32]:
# Measles vaccination status Based on health card recorde with date
raw.data <- raw.data |> 
		mutate(measles = car::recode(H9, 'c(1, 2, 3) = 1;
                               0 = 0;
                               8 = 0'))

count(raw.data, measles)

# A tibble: 3 × 2
  measles                           n
  <dbl+lbl>                     <int>
1  0 [No]                        2668
2  1 [Vaccination date on card]  2722
3 NA                             3899

### **Child's age when recieved vaccination (health card based)**

In [5]:
# Date of vaccination - Moz DHS 2022-23
vaccines_list <- with(raw.data, list(
  list(name = "bcg",     year = H2Y,  month = H2M,  day = H2D),
  list(name = "polio0",  year = H0Y,  month = H0M,  day = H0D),
  list(name = "polio1",   year = H4Y,  month = H4M,  day = H4D),
  list(name = "polio2",   year = H6Y,  month = H6M,  day = H6D),
  list(name = "polio3",   year = H8Y,  month = H8M,  day = H8D),
	list(name = "penta1",  year = H51Y, month = H51M, day = H51D),
  list(name = "penta2",  year = H52Y, month = H52M, day = H52D),
  list(name = "penta3",  year = H53Y, month = H53M, day = H53D),
  list(name = "pcv1",    year = H54Y, month = H54M, day = H54D),
  list(name = "pcv2",    year = H55Y, month = H55M, day = H55D),
  list(name = "pcv3",    year = H56Y, month = H56M, day = H56D),
  list(name = "rota1",   year = H57Y, month = H57M, day = H57D),
  list(name = "rota2",   year = H58Y, month = H58M, day = H58D),
  list(name = "ipv",     year = H60Y, month = H60M, day = H60D),
  list(name = "measles1",year = H9Y,  month = H9M,  day = H9D),
  list(name = "measles2",year = H9AY, month = H9AM, day = H9AD),
  list(name = "vitA1",   year = H33Y, month = H33M, day = H33D)
))


In [6]:
	# Remove unplausible years "9998 [Don't know]"
	fix_year <- function(year){
		return(ifelse(year > 2023, NA, year))
	}

In [7]:
# Create birth date and interview date
birth_and_interview_dates <- function(data){
	data |>
		mutate(
						date_birth = as.Date(ifelse(is.na(fix_year(B2)), NA, paste0(B2,"-",B1,"-",B17))),
						date_interview = as.Date(ifelse(is.na(fix_year(V007)), NA, paste0(V007,"-",V006,"-",V016))),
		)
}

In [8]:
# Compute child's age of vaccination (each vaccine) 
# Compute the variable that indicate whether child vaccinated after 9 months
# 9 months after measles 1 elegible age 
vac_age <- function(data, vac_list){
	
			for(vac in vac_list){
				date_vac = paste0(vac$name, "_date")
 				age_vac = paste0(vac$name, "_age")
				age_vac_after_9 = paste0(vac$name, "_after_9")
				
				data[[date_vac]] = as.Date(ifelse(is.na(fix_year(vac$year)), NA, 
																					paste0(vac$year,"-",vac$month, "-", vac$day)))
				
				data[[age_vac]] = ceiling(unlist(data[date_vac]-data["date_birth"])/30)
				data[age_vac_after_9] = ifelse(data[[age_vac]] >= 9, 1, 0)
			}
	
	data[["vac_after_9"]] <- with(data, ifelse( bcg_after_9 == 1  |
																							polio0_after_9==1 |
																							polio1_after_9==1 |
																							polio2_after_9==1 |
																							polio3_after_9==1 |
																							penta1_after_9==1 |
																							penta2_after_9==1 |
																							penta3_after_9==1 |
																							pcv1_after_9==1   |  
																							pcv2_after_9==1   |  
																							pcv3_after_9==1   |  
																							rota1_after_9==1  | 
																							rota2_after_9==1  | 
																							ipv_after_9==1    |   
																							#measles1_after_9==1 |
																							measles2_after_9==1 |
																							vitA1_after_9==1, 1, 0)  

)
	return(data)
}

In [33]:
# call 
raw.data_1 <- birth_and_interview_dates(raw.data)

In [34]:
# call
raw.data_2 <- vac_age(data = raw.data_1, vac_list = vaccines_list)

### **Measles MOV**

In [35]:
raw.data_2 <- raw.data_2 |> mutate(mov = ifelse(measles == 0 & vac_after_9 == 1 & H1 == 1, 1, 0))
count(raw.data_2, mov)

# A tibble: 3 × 2
    mov     n
  <dbl> <int>
1     0  3825
2     1   229
3    NA  5235

### **Unweighted measles MOV estimates for under 5 children**

In [36]:
# survey object - simple random sampling
data_unwt <- svydesign(ids = ~1, weights = ~1, data = raw.data_2)

# Estimates
mov <- svyciprop(formula = ~mov, method = "logit", design = data_unwt) 

In [37]:
paste0(round(as.numeric(mov)*100,2), " [", round(attr(mov, "ci")[1] *100, 1), "-", round(attr(mov, "ci")[2] * 100, 2),"]")

[1] "5.65 [5-6.4]"

In [44]:
svyby(formula = ~mov,by = ~V024, FUN=svyciprop, method="beta", na.rm=T, vartype = "ci", design = data_unwt)  * 100

   V024        mov       ci_l      ci_u
1   100  4.1916168 2.61301906  6.336302
2   200  6.2146893 4.31606088  8.617486
3   300 10.6299213 8.08680620 13.641724
4   400  2.8061224 1.40889190  4.965521
5   500  6.3291139 4.13728184  9.201681
6   600  6.6210046 4.47857626  9.370864
7   700  6.2500000 4.00274860  9.230915
8   800  4.1493776 2.00734682  7.498510
9   900  3.2608696 1.50163579  6.099881
10 1000  6.0185185 3.24306686 10.072235
11 1100  0.5319149 0.01346143  2.928070